In [ ]:
#장기수익창출(Risk parity -> MVO)
import pandas as pd
from risk_parity import risk_parity_allocation 
from block_definitions import classify_assets_by_blocks 
from mvo import mvo_optimizer 

# 데이터 로드
data_path = '/Users/heunjjang/Desktop/sample.csv'
data = pd.read_csv(data_path)

l1_assets, l2_indices, l3_markets, l4_etfs = classify_assets_by_blocks()

# 사용자 입력
goal_period = int(input("투자기간 (년): "))
if goal_period < 10:
    raise ValueError("장기수익창출 포트폴리오는 10년 이상의 투자기간이 필요합니다. 투자기간을 수정하세요.")
risk_preference = int(input("위험 선호도 (1: 매우 안전, 5: 매우 공격적): "))
risk_aversion = 6 - risk_preference  

# 단계별 실행
final_allocations = {}
upper_weights = {"L1": 1.0}

# L1 단계: Risk Parity
final_allocations["L1"] = risk_parity_allocation(data[l1_assets], risk_aversion)
# L2 -> L4 MVO
for level, assets, bounds, prev_level in [
    ("L2", l2_indices, (0.05, 0.5), "L1"), 
    ("L3", l3_markets, (0.05, 0.5), "L2"),
    ("L4", l4_etfs, (0.05, 0.4), "L3")
]:
    upper_weight = sum(final_allocations[prev_level].values())
    cov_matrix = data[assets].pct_change().dropna().cov().values
    expected_returns = data[assets].pct_change().dropna().mean().values

    weights = mvo_optimizer(
        nodes=assets,
        covariance_matrix=cov_matrix,
        expected_returns=expected_returns,
        weight_bounds=[bounds] * len(assets),
        risk_aversion=risk_aversion
    )

    # 하위 단계에 상위 비중 적용
    final_allocations[level] = {asset: weight * upper_weight for asset, weight in zip(assets, weights)}

l4_allocation_list = [f"{asset}: {weight * 100:.2f}%" for asset, weight in final_allocations["L4"].items()]
print("\n### 장기수익창출 포트폴리오 ###")
print(l4_allocation_list)



### 장기수익창출 포트폴리오 ###
['REET: 5.00%', 'USO: 5.00%', 'MTUM: 10.47%', 'SPLV: 11.75%', 'SRVR: 5.00%', 'XHB: 15.67%', 'SLV: 22.47%', 'JPST: 24.65%']
